# Sıralama ve Pozisyon Boyutlandırma

Bu not defterinde, **PyBroker**'in işlem stratejinizdeki hisse senedi sembollerini sıralamanıza ve bir grup sembol için konum boyutlarını ayarlamanıza olanak tanıyan özelliklerini öğreneceğiz. Bu özellikler sayesinde stratejinizi kolayca optimize edebilir ve riski daha etkin bir şekilde yönetebilirsiniz.

In [1]:
import pybroker
from pybroker import Strategy, StrategyConfig, YFinance

pybroker.enable_data_source_cache('ranking_and_pos_sizing')

## Sıralama İşaretçisi Sembolleri

Bu bölümde alış emirleri verirken hisse senedi sembollerinin nasıl sıralanacağını öğreneceğiz. Satın alma emirleri verirken hisse senedi sembollerinin hacme göre nasıl sıralanacağına dair bir örnekle başlayalım.

In [2]:
def buy_highest_volume(ctx):
    # İşlem gören tüm hisse senetlerinde uzun pozisyon yoksa:
    if not tuple(ctx.long_positions()):
        ctx.buy_shares = ctx.calc_target_shares(1)
        ctx.hold_bars = 2
        ctx.score = ctx.volume[-1]

```buy_highest_volume``` işlevi, hisse senedi sembollerini en son işlem hacimlerine göre sıralar ve portföyün %100'ünü 2 bara ayırır. ```ctx.score```, en son işlem hacmi olan ```ctx.volume[-1]``` olarak ayarlanmıştır.

In [3]:
config = StrategyConfig(max_long_positions=1)
strategy = Strategy(YFinance(), '6/1/2021', '6/1/2022', config)
strategy.add_execution(buy_highest_volume, ['T', 'F', 'GM', 'PFE'])

Herhangi bir zamanda tutulabilecek uzun pozisyon sayısını ```1``` ile sınırlamak için [max_long_positions](https://www.pybroker.com/en/latest/reference/pybroker.config.html) ayarladık. #pybroker.config.StrategyConfig.max_long_positions) değerini [StrategyConfig](https://www.pybroker.com/en/latest/reference/pybroker.config.html#pybroker.config.StrategyConfig) içinde ```1``` olarak değiştirin ). Bu örnekte, ``buy_highest_volume`` işlevini [Strategy](https://www.pybroker.com/en/latest/reference/pybroker.strategy.html#pybroker.strategy.Strategy) nesnesine ekliyoruz. ve takas edilecek hisse senedi sembollerini belirtin: ```['T', 'F', 'GM', 'PFE']```.

In [4]:
result = strategy.backtest()
result.trades

Backtesting: 2021-06-01 00:00:00 to 2022-06-01 00:00:00

Loading bar data...
[*********************100%***********************]  4 of 4 completed
Loaded bar data: 0:00:00 

Test split: 2021-06-01 00:00:00 to 2022-05-31 00:00:00


  0% (0 of 253) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--
 59% (151 of 253) |#############         | Elapsed Time: 0:00:00 ETA:  00:00:00
100% (253 of 253) |######################| Elapsed Time: 0:00:00 Time:  0:00:00



Finished backtest: 0:00:03


,type,symbol,entry_date,exit_date,entry,exit,shares,pnl,return_pct,agg_pnl,bars,pnl_per_bar,stop
id,,,,,,,,,,,,,
1,long,F,2021-06-02,2021-06-04,14.85,16.13,6734,8619.52,8.62,8619.52,2,4309.76,bar
2,long,F,2021-06-07,2021-06-09,15.93,15.51,6801,-2856.42,-2.64,5763.10,2,-1428.21,bar
3,long,F,2021-06-10,2021-06-14,15.43,15.06,6832,-2527.84,-2.40,3235.26,2,-1263.92,bar
4,long,F,2021-06-15,2021-06-17,14.96,14.99,6900,207.00,0.20,3442.26,2,103.50,bar
5,long,F,2021-06-18,2021-06-22,14.61,14.96,7003,2451.05,2.40,5893.31,2,1225.53,bar
...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,long,F,2022-05-10,2022-05-12,13.43,12.47,7263,-6972.48,-7.15,-9423.13,2,-3486.24,bar
81,long,F,2022-05-13,2022-05-17,13.25,13.34,6835,615.15,0.68,-8807.98,2,307.58,bar
82,long,F,2022-05-18,2022-05-20,13.03,12.59,6739,-2965.16,-3.38,-11773.14,2,-1482.58,bar


## Konum Boyutlarını Ayarlama

**PyBroker**'da konum boyutlarını birden fazla onay işaretine göre ayarlayabilirsiniz. Bunu göstermek için, 100 gün sonra işlem görmeye başlayan ve pozisyonları 30 gün boyunca tutan basit bir satın alma ve tutma stratejisini ele alalım:

In [5]:
def buy_and_hold(ctx):
    if not ctx.long_pos() and ctx.bars >= 100:
        ctx.buy_shares = 100
        ctx.hold_bars = 30
        
strategy = Strategy(YFinance(), '6/1/2021', '6/1/2022')
strategy.add_execution(buy_and_hold, ['T', 'F', 'GM', 'PFE'])

Bu, ```['T', 'F', 'GM', 'PFE']``` her birinden ```100``` hisse satın alacak. Peki ya eşit konum boyutlandırmayı kullanmak istemiyorsanız? Örneğin, portföyün genel oynaklığını azaltmak için daha düşük oynaklığa sahip hisse senetlerine daha fazla hisse tahsis edilecek şekilde pozisyonları boyutlandırmak isteyebilirsiniz.

Her bir şerit için konum boyutlandırmasını özelleştirmek amacıyla, şu değeri hesaplayan bir [pos_size_handler](https://www.pybroker.com/en/latest/reference/pybroker.strategy.html#pybroker.strategy.Strategy.set_pos_size_handler) işlevi tanımlayabiliriz. her bir şerit için konum boyutu:

In [6]:
import numpy as np

def pos_size_handler(ctx):
    # Tüm satın alma sinyallerini alın.
    signals = tuple(ctx.signals("buy"))
    # Alış sinyali yoksa (yani yalnızca satış sinyali varsa) geri dönün.
    if not signals:
        return
    # Volatilitenin şu şekilde tanımlandığı ters volatiliteyi hesaplar:
    # Son 100 gündeki kapanış fiyatlarının standart sapması.
    get_inverse_volatility = lambda signal: 1 / np.std(signal.bar_data.close[-100:])
    # Tüm satın alma sinyallerinin ters volatilitelerini toplar.
    total_inverse_volatility = sum(map(get_inverse_volatility, signals))
    for signal in signals:
        size = get_inverse_volatility(signal) / total_inverse_volatility
        # En son kapanış fiyatına göre hisse sayısını hesaplayın.
        shares = ctx.calc_target_shares(size, signal.bar_data.close[-1], cash=95_000)
        ctx.set_shares(signal, shares)
        
strategy.set_pos_size_handler(pos_size_handler)

In [7]:
result = strategy.backtest()

Backtesting: 2021-06-01 00:00:00 to 2022-06-01 00:00:00

Loaded cached bar data.

Test split: 2021-06-01 00:00:00 to 2022-05-31 00:00:00


  0% (0 of 253) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--
 59% (151 of 253) |#############         | Elapsed Time: 0:00:00 ETA:  00:00:00
 95% (241 of 253) |####################  | Elapsed Time: 0:00:00 ETA:   0:00:00
100% (253 of 253) |######################| Elapsed Time: 0:00:00 Time:  0:00:00



Finished backtest: 0:00:00


In [8]:
result.trades

,type,symbol,entry_date,exit_date,entry,exit,shares,pnl,return_pct,agg_pnl,bars,pnl_per_bar,stop
id,,,,,,,,,,,,,
1,long,F,2021-10-21,2021-12-03,16.41,19.66,2028,6591.00,19.80,6591.00,30,219.70,bar
2,long,PFE,2021-10-21,2021-12-03,42.76,53.75,235,2582.65,25.70,9173.65,30,86.09,bar
3,long,T,2021-10-21,2021-12-03,19.60,17.54,2248,-4630.88,-10.51,4542.77,30,-154.36,bar
4,long,GM,2021-10-21,2021-12-03,58.20,60.28,129,268.32,3.57,4811.09,30,8.94,bar
5,long,F,2021-12-06,2022-01-19,19.05,23.66,1075,4955.75,24.20,9766.84,30,165.19,bar
6,long,PFE,2021-12-06,2022-01-19,52.57,53.97,290,406.00,2.66,10172.84,30,13.53,bar
7,long,T,2021-12-06,2022-01-19,17.81,20.49,2665,7142.20,15.05,17315.04,30,238.07,bar
8,long,GM,2021-12-06,2022-01-19,59.72,57.99,209,-361.57,-2.90,16953.47,30,-12.05,bar
9,long,F,2022-01-20,2022-03-04,22.22,17.01,789,-4110.69,-23.45,12842.78,30,-137.02,bar


Bu yöntemin kullanılması, portföy tahsislerini belirlemek için [Ortalama-Varyans Optimizasyonu](https://en.wikipedia.org/wiki/Modern_portfolio_theory) kullanılması gibi birçok olasılığa olanak tanır.
